<a href="https://colab.research.google.com/github/UniVR-DH/DBMS-course/blob/main/notebooks/lab02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQLite Exercise on CREATE, ALTER, DELETE, SELECT Queries

We are going to create some tables, test with different functions for the constraints, test the alter table command as well.

You can compare the behaviour of another two systems (MySQL and Postgres) here https://www.db-fiddle.com/

In [1]:
import sqlite3
conn = sqlite3.connect("exams.db")
cursor = conn.cursor()

In [2]:
cursor.execute('''
CREATE TABLE student (
   studid INTEGER PRIMARY KEY,
   name VARCHAR(50),
   semester INTEGER
);
''')

cursor.execute('''
CREATE TABLE professor (
   id INTEGER PRIMARY KEY,
   name VARCHAR(50)
);
''')


#-- Insert the data
cursor.execute('''
INSERT INTO student (studid, name, semester) VALUES
   (24002, 'Xenokrates', 18),
   (25403, 'Jonas', 12),
   (26120, 'Fichte', 10),
   (26830, 'Aristoxenos', 8),
   (27550, 'Schopenhauer', 6),
   (28106, 'Carnap', 3),
   (29120, 'Theophrastos', 2),
   (29555, 'Feuerbach', 12),
   (30000, 'Fe', 3);
''')


cursor.execute('''
-- Insert professors data
INSERT INTO professor (id, name) VALUES
   (2125, 'Socrates'),
   (2126, 'Russel'),
   (2127, 'Kant');
''')

conn.commit()
print("Ready!")

Ready!


Insert other two professors "Susan" and "Marie"

## Autogenerate values for primary keys:

The following command is standard SQL **but** is not valid in SQLite -- it works in Postgres

```
CREATE SEQUENCE course_seq START WITH 945;

CREATE TABLE course (
     id INTEGER PRIMARY KEY DEFAULT nextval('course_seq'),
   title VARCHAR(50)
);
```


SQLite does not support `SEQUENCE`, use `AUTOINCREMENT` see code in  the next cell


Note that SQLite has a "ancillary" table called `sqlite_sequence` where it keeps info about these "number generators"


In [3]:
cursor.execute('''
CREATE TABLE course (
     id INTEGER PRIMARY KEY AUTOINCREMENT,
   title VARCHAR(50)
);
''')

# Note: In SQLite, if you need to start the AUTOINCREMENT
# from a specific number like 2125 or 945, you'll need to
# modify the sqlite_sequence table after creating your tables:
cursor.execute('''
INSERT INTO sqlite_sequence(seq, name) VALUES (  944 , 'course');
''')

# If you do not use the command above, then the AUTOINCREMENT
# will simply start from 1

conn.commit()
print("Ready!")

Ready!


In [4]:
cursor.execute("SELECT name, seq FROM sqlite_sequence")
rows = cursor.fetchall()

for row in rows:
  print(row)

('course', 944)


**Inserting courses by specifying only the title is possible because we have declared the primary  key to be an autoincrement**

In [5]:
cursor.execute('''
INSERT INTO course (title) VALUES
   ('Basics'),
   ('History'),
   ('Ethics');
''')
conn.commit()
print("Ready!")

Ready!


**In the box above we need to specify the attribute name, if we skip it we get an error!  See below**

In [6]:
# we write course VALUES instead of course(title) VALUES and we get an error
cursor.execute('''
INSERT INTO course VALUES
   ('Basics'),
   ('History'),
   ('Ethics');
''')
conn.commit()
print("Ready!")

OperationalError: table course has 2 columns but 1 values were supplied

**Let's see the effect of AUTOINCREMENT**

In [7]:
cursor.execute("SELECT name, seq FROM sqlite_sequence")
rows = cursor.fetchall()

for row in rows:
  print(row)

('course', 947)


In [8]:
cursor.execute("SELECT * FROM course")
rows = cursor.fetchall()

for row in rows:
  print(row)

(945, 'Basics')
(946, 'History')
(947, 'Ethics')


Despite the autoincrement we can still manually specify the value ofr the primary key... but **the system has extra work to do!**

In [9]:
cursor.execute('''
INSERT INTO course(id, title) VALUES
   ('10', 'Adv. Basics'),
   ('11', 'Adv. History'),
   ('12', 'Adv. Ethics');
''')
conn.commit()
print("Ready!")

Ready!


In [10]:
cursor.execute("SELECT * FROM course")
rows = cursor.fetchall()

for row in rows:
  print(row)

(10, 'Adv. Basics')
(11, 'Adv. History')
(12, 'Adv. Ethics')
(945, 'Basics')
(946, 'History')
(947, 'Ethics')


In [11]:
cursor.execute("SELECT name, seq FROM sqlite_sequence")
rows = cursor.fetchall()

for row in rows:
  print(row)

('course', 947)


**Look at the following command, what is going to happen?**

In [12]:
cursor.execute('''
INSERT INTO course(id, title) VALUES
   ('948', 'English 1'),
   ('949', 'French 1');
''')
conn.commit()
print("Ready!")

Ready!


In [13]:
cursor.execute("SELECT name, seq FROM sqlite_sequence")
rows = cursor.fetchall()

for row in rows:
  print(row)

('course', 949)


In [14]:
cursor.execute("SELECT * FROM course")
rows = cursor.fetchall()

for row in rows:
  print(row)

(10, 'Adv. Basics')
(11, 'Adv. History')
(12, 'Adv. Ethics')
(945, 'Basics')
(946, 'History')
(947, 'Ethics')
(948, 'English 1')
(949, 'French 1')


**...and now??**

In [16]:
cursor.execute('''
INSERT INTO course (title) VALUES
   ('English 2'),
   ('French 2');
''')
conn.commit()
print("Ready!")

Ready!


In [17]:
cursor.execute("SELECT * FROM course")
rows = cursor.fetchall()

for row in rows:
  print(row)

(10, 'Adv. Basics')
(11, 'Adv. History')
(12, 'Adv. Ethics')
(945, 'Basics')
(946, 'History')
(947, 'Ethics')
(948, 'English 1')
(949, 'French 1')
(950, 'English 2')
(951, 'French 2')
(952, 'English 2')
(953, 'French 2')


## ALTER TABLE Test


In [18]:
cursor.execute('''
ALTER TABLE course ADD COLUMN ects INT;
''')

cursor.execute("SELECT * FROM course")
rows = cursor.fetchall()

for row in rows:
  print(row)
conn.commit()
print("Ready!")

(10, 'Adv. Basics', None)
(11, 'Adv. History', None)
(12, 'Adv. Ethics', None)
(945, 'Basics', None)
(946, 'History', None)
(947, 'Ethics', None)
(948, 'English 1', None)
(949, 'French 1', None)
(950, 'English 2', None)
(951, 'French 2', None)
(952, 'English 2', None)
(953, 'French 2', None)
Ready!


**Note: in SQLite ALTER COLUMN is not supported**

The following command will not work in SQLite

```
ALTER TABLE course ALTER COLUMN ects SET NOT NULL;
```
And so we cannot add constraints, we need to create a new column and drop the old column


In [19]:
cursor.execute('''
ALTER TABLE course ADD COLUMN ects_no_null integer NOT NULL DEFAULT 2;
''')

cursor.execute('''
ALTER TABLE course DROP COLUMN ects;
''')


cursor.execute("SELECT * FROM course")
rows = cursor.fetchall()

for row in rows:
  print(row)
conn.commit()
print("Ready!")

(10, 'Adv. Basics', 2)
(11, 'Adv. History', 2)
(12, 'Adv. Ethics', 2)
(945, 'Basics', 2)
(946, 'History', 2)
(947, 'Ethics', 2)
(948, 'English 1', 2)
(949, 'French 1', 2)
(950, 'English 2', 2)
(951, 'French 2', 2)
(952, 'English 2', 2)
(953, 'French 2', 2)
Ready!


## Exercise 01

Create a table called `takes`  with two attributes studid and courseid to connect students to courses.

In [20]:
# Create the takes table
cursor.execute('''
CREATE TABLE takes (
    studid INTEGER,
    courseid INTEGER,
    FOREIGN KEY (studid) REFERENCES student(studid),
    FOREIGN KEY (courseid) REFERENCES course(id)
);
''')

conn.commit()
print("Ready!")

Ready!


Then, insert into  takes the tuples to register all students to the course with id 946

In [21]:
# Insert tuples to register all students to the course with id 946
cursor.execute('''
INSERT INTO takes (studid, courseid)
SELECT studid, 946 FROM student;
''')

conn.commit()
print("Ready!")

Ready!


Then, insert into  takes the tuples to register all students with semester above 5 to the course with id *948*

In [22]:
cursor.execute('''

--- ????


''')

Write the SQL query to return all the course ids for student with name 'Jonas'

In [23]:
cursor.execute('''
  SELECT t.courseid
  FROM takes AS t
  JOIN student AS s ON t.studid = s.studid
  WHERE s.name = 'Jonas';
''')

for row in cursor.fetchall():
  print(row[0])


946


*Write the SQL query to return all the course ids for student with name starting ith 'F'*

In [ ]:
cursor.execute('''

--- ????


''')

rows = cursor.fetchall()

for row in rows:
  print(row[0])

## Exercise 02


Create a table called `teaches` then insert the data to say that Socrates teaches Basic and French, and Russel teaches Ethics


In [24]:
# Create the teaches table
cursor.execute('''
CREATE TABLE teaches (
    profid INTEGER,
    courseid INTEGER,
    FOREIGN KEY (profid) REFERENCES professor(id),
    FOREIGN KEY (courseid) REFERENCES course(id)
);
''')

# Insert data for Socrates teaching Basic and French
cursor.execute('''
INSERT INTO teaches (profid, courseid)
SELECT p.id, c.id
FROM professor p, course c
WHERE p.name = 'Socrates' AND c.title IN ('Basics', 'French 1');
''')

# Insert data for Russel teaching Ethics
cursor.execute('''
INSERT INTO teaches (profid, courseid)
SELECT p.id, c.id
FROM professor p, course c
WHERE p.name = 'Russel' AND c.title = 'Ethics';
''')

conn.commit()
print("Ready!")

Ready!


Provide the title of the courses taught by professor with id 2126

In [ ]:
cursor.execute('''

--- ????


''')

rows = cursor.fetchall()

for row in rows:
  print(row[0])

Provide the title of the courses taught by professor with name starting with 'F'